# Data reclassification

Reclassifying data based on specific criteria is a common task when doing GIS analysis. The purpose of this lesson is to see how we can reclassify values based on some criteria which can be whatever, such as:

```
1. if rent burden is > 50 %

    AND

    2. % poverty is greater than 50 %

    ------------------------------------------------------

    IF TRUE: ==> Household receives Housing Choice Vouchers
    IF NOT TRUE: ==> Household can rent market-rate housing
```

In this tutorial, we will:

1. Use classification schemes from the PySAL [mapclassify library](https://pysal.org/mapclassify/) to classify travel times into multiple classes.

2. Create a custom classifier to classify demographic and socioeconomic variables in order to find out priority locations for housing and transit policy interventions:
   - Majority non-White residents
   - High public transit reliance
   - High rent burden
   - High poverty rates

## Input data

We will use [four tables from the US Census](https://data.census.gov/) that contains race/ethnicity, rent burden, percent transit users, and poverty rates.

These tables are:
1. [S0601: Selected Characteristics of the Total and Native Populations in the United States](https://data.census.gov/table?q=s0601)
2. [S0701: Poverty Status in the Past 12 Months](https://data.census.gov/table?q=poverty&g=050XX00US06037$1400000&y=2023) 
3. [S0802: Means of Transportation to Work by Selected Characteristics](https://data.census.gov/table?q=s0802)
4. [B25070: Gross Rent as a Percentage of Household Icome in the Past 12 Month](https://data.census.gov/table?q=b25070&g=050XX00US06037$1400000&y=2023)

In this tutorial, we will use a census tract shapefile from [Tiger/Line](https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2023&layergroup=Census+Tracts) to merge the data table that has been cleaned up in Excel:
`"data/tl_2023_06_tract.zip"`






## Common classifiers

### Classification schemes for thematic maps


[PySAL](http://pysal.readthedocs.io/en/latest) -module is an extensive Python library for spatial analysis. It also includes all of the most common data classifiers that are used commonly e.g. when visualizing data. Available map classifiers in [pysal's mapclassify -module](https://pysal.readthedocs.io/en/v1.11.0/library/esda/mapclassify.html):

 - Box_Plot
 - Equal_Interval
 - Fisher_Jenks
 - Fisher_Jenks_Sampled
 - HeadTail_Breaks
 - Jenks_Caspall
 - Jenks_Caspall_Forced
 - Jenks_Caspall_Sampled
 - Max_P_Classifier
 - Maximum_Breaks
 - Natural_Breaks
 - Quantiles
 - Percentiles
 - Std_Mean
 - User_Defined

- First, we need to read our zipped shapefiles for California Tracts and LA County:

In [ ]:
import geopandas as gpd

caTracts = "data/tl_2023_06_tract.zip"

# Read the GeoJSON file similarly as Shapefile
tracts = gpd.read_file(caTracts)

# Let's see what we have
tracts.head()

In [ ]:
# import county boundary to clip tracts from state to LA County boundary
county = "data/County_Boundary.zip"

clip = gpd.read_file(county)

# project tracts and clip to same coordinate system
tracts = tracts.to_crs(epsg=3857)
clip = clip.to_crs(epsg=3857)


# Clipping with geopandas 'clip' function
## Clipping is commonly referred to as the 'cookie cutter' geospatial tool
With geopandas.clip, we can cut (or select) census tracts for the whole state of California to the LA County boundary. 

![An example of the clip tool](images/clip.png) from {GIS Geography](https://gisgeography.com/clip-tool-gis/)

In [ ]:
# Clip the tracts to the county boundary
la_tracts = gpd.clip(tracts, clip)

# Let's see what we have
la_tracts.head()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt 

# Plot la_tracts and clip
fig, ax = plt.subplots(figsize=(10, 10))

# Plot LA tracts with opaque grey fill and dark grey skinny edge color
la_tracts.plot(ax=ax, color='grey', edgecolor='darkgrey', alpha=0.5, linewidth=0.75)

# Plot the clip boundary with black color
clip.boundary.plot(ax=ax, color='black', linewidth=1)

plt.title('LA Tracts and County Boundary')
plt.show()

- Second, we need to import our CSV and format the FIPS code to contain the leading zero.

In [ ]:
import pandas as pd

census = "data/acs2023_LA.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(census, dtype={'GEOID': str})

# Print the header of the CSV
df.head()

In [ ]:
# Add leading zeros to GEOID column
df['GEOID'] = df['GEOID'].str.zfill(11)

# Print the head of the dataframe to verify the changes
df.head()

As we can see, there are 4 different variables that we are interested in are columns called `Pct_NonWhite`, `Pct_PublicTrans`, `Pct_RentBurden`, `Pct_BelowPov ` which will determine our Housing and Transit Vulnerability Index.

**The NoData values are presented with value -1**. 

- Thus we need to remove the No Data values first.


In [ ]:
# Include only data that is above or equal to 0
df = df.loc[df['Pct_NonWhite'] >=0]
df = df.loc[df['Pct_PublicTrans'] >=0]
df = df.loc[df['Pct_RentBurden'] >=0]
df = df.loc[df['Pct_BelowPov'] >=0]

# Merging the data

- Let's merge the data on FIPS 
- Let's plot the data and see how it looks like
- `cmap` parameter defines the color map. Read more about [choosing colormaps in matplotlib](https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html)
- `scheme` option scales the colors according to a classification scheme (requires `mapclassify` module to be installed):

In [ ]:
# Merge the dataframes on GEOID
merged_tracts = la_tracts.merge(df, left_on='GEOID', right_on='GEOID')

# Print the head of the merged dataframe
print(merged_tracts.head())

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Set fig, ax and plot with legend outside of plot
fig, ax = plt.subplots(figsize=(10, 10))
merged_tracts.plot(column="Pct_NonWhite", scheme="Natural_Breaks", k=5, cmap="YlGn", linewidth=0, legend=True, ax=ax)

# Use tight layout
plt.tight_layout()
plt.title('Percentage of Non-White Residents in LA County')

ax.axis('off')

# Place legend outside of plot
ax.get_legend().set_bbox_to_anchor((1.5, 1))


As we can see from this map, there are particular locations in LA County where there are higher rates of non-White residents.

- Let's also make a plot about Public Transit use, Rent Burden and Percent Below Poverty:

In [ ]:
# Plot using 5 classes and classify the values using "Natural Breaks" classification
fig, ax = plt.subplots(figsize=(10, 10))
merged_tracts.plot(column="Pct_PublicTrans", scheme="Natural_Breaks", k=5, cmap="YlGn", linewidth=0, legend=True, ax=ax)

# Use tight layout
plt.tight_layout()
plt.title('Percentage of Residents taking Public Transit in LA County')

ax.axis('off')

# Place legend outside of plot
ax.get_legend().set_bbox_to_anchor((1.5, 1))

In [ ]:
# Plot using 5 classes and classify the values using "Natural Breaks" classification
fig, ax = plt.subplots(figsize=(10, 10))
merged_tracts.plot(column="Pct_RentBurden", scheme="Natural_Breaks", k=5, cmap="YlGn", linewidth=0, legend=True, ax = ax)

# Use tight layout
plt.tight_layout()
plt.title('Percentage of Rent Burdened Residents  in LA County')

ax.axis('off')

# Place legend outside of plot
ax.get_legend().set_bbox_to_anchor((1.5, 1))

In [ ]:
# Plot using 5 classes and classify the values using "Natural Breaks" classification
fig, ax = plt.subplots(figsize=(10, 10))
merged_tracts.plot(column="Pct_BelowPov", scheme="Natural_Breaks", k=5, cmap="YlGn", linewidth=0, legend=True, ax = ax)

# Use tight layout
plt.tight_layout()
plt.title('Percentage of  Residents living in Poverty in LA County')

ax.axis('off')

# Place legend outside of plot
ax.get_legend().set_bbox_to_anchor((1.5, 1))

# Merging the data through reclassification
Okay, from here we can see where our demographic and socioeconomic variables are concentrating, but we want to make a vulnerability index by combining them together. 

### Applying classifiers to data

As mentioned, the `scheme` option defines the classification scheme using `pysal/mapclassify`. Let's have a closer look at how these classifiers work.

In [ ]:
import mapclassify

- Natural Breaks

In [ ]:
mapclassify.NaturalBreaks(y=merged_tracts['Pct_NonWhite'], k=5)

- Quantiles (default is 5 classes):

In [ ]:
mapclassify.Quantiles(y=merged_tracts['Pct_NonWhite'])

- It's possible to extract the threshold values into an array:

In [ ]:
classifier = mapclassify.NaturalBreaks(y=merged_tracts['Pct_NonWhite'], k=5)
classifier.bins

- Let's apply one of the `Pysal` classifiers into our data and classify the travel times by non-White residents into 5 classes
- The classifier needs to be initialized first with `make()` function that takes the number of desired classes as input parameter

In [ ]:
# Create a Natural Breaks classifier
classifier = mapclassify.NaturalBreaks.make(k=5)

- Now we can apply that classifier into our data by using `apply` -function

In [ ]:
# Classify the data
classifications = merged_tracts[['Pct_NonWhite']].apply(classifier)

# Let's see what we have
classifications.head()

In [ ]:
type(classifications)

Okay, so now we have a DataFrame where our input column was classified into 5 different classes (numbers 1-5) based on [Natural Breaks classification](http://wiki-1-1930356585.us-east-1.elb.amazonaws.com/wiki/index.php/Jenks_Natural_Breaks_Classification).

- We can also add the classification values directly into a new column in our dataframe:

In [ ]:
# Rename the column so that we know that it was classified with natural breaks
merged_tracts['nonWhite_class'] = merged_tracts[['Pct_NonWhite']].apply(classifier)

# Check the original values and classification
merged_tracts[['nonWhite_class', 'Pct_NonWhite']].head()

Great, now we have those values in our accessibility GeoDataFrame. Let's visualize the results and see how they look.

In [ ]:
# Plot
merged_tracts.plot(column="nonWhite_class", linewidth=0, legend=True)

# Use tight layout
plt.tight_layout()
plt.title('Classified Non-White Residents in LA County')
plt.set_cmap('YlGn')

And here we go, now we have a map where we have used one of the common classifiers to classify our data into 5 classes.

## Plotting a histogram

A histogram is a graphic representation of the distribution of the data. When classifying the data, it's always good to consider how the data is distributed, and how the classification shceme divides values into different ranges. 

- plot the histogram using [pandas.DataFrame.plot.hist](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.hist.html)
- Number of histogram bins (groups of data) can be controlled using the parameter `bins`:

In [ ]:
# Histogram for non-white residents
merged_tracts['Pct_NonWhite'].plot.hist(bins=50)

Let's also add threshold values on thop of the histogram as vertical lines.

- Natural Breaks:

In [ ]:
# Define classifier
classifier = mapclassify.NaturalBreaks(y=merged_tracts['Pct_NonWhite'], k=5)

# Plot histogram for public transport rush hour travel time
merged_tracts['Pct_NonWhite'].plot.hist(bins=50)

# Add vertical lines for class breaks
for value in classifier.bins:
    plt.axvline(value, color='k', linestyle='dashed', linewidth=1)

- Quantiles:

In [ ]:
# Define classifier
classifier = mapclassify.Quantiles(y=merged_tracts['Pct_NonWhite'])

# Plot histogram for public transport rush hour travel time
merged_tracts['Pct_NonWhite'].plot.hist(bins=50)

for value in classifier.bins:
    plt.axvline(value, color='k', linestyle='dashed', linewidth=1)


<div class="alert alert-info">

**Task**

Select another column from the data (for example, poverty: `Pct_BelowPov`). Do the following visualizations using one of the classification schemes available from [pysal/mapclassify](https://github.com/pysal/mapclassify):
    
- histogram with vertical lines showing the classification bins
- thematic map using the classification scheme


</div>

In [ ]:
# Export merged_tracts to a CSV file
merged_tracts.to_csv('merged_tracts.csv', index=False)

## Creating a vulnerability index

**Adding the columns together to get a priority score**

Let's create three new columns, add them together, and create our Housing and Transit Vulnerability Index. We can then plot this with matplotlib and folium.


In [ ]:
# Define classifiers for each column
classifier_public_trans = mapclassify.NaturalBreaks(y=merged_tracts['Pct_PublicTrans'], k=5)
classifier_rent_burden = mapclassify.NaturalBreaks(y=merged_tracts['Pct_RentBurden'], k=5)
classifier_below_pov = mapclassify.NaturalBreaks(y=merged_tracts['Pct_BelowPov'], k=5)

# Apply classifiers and create new columns
merged_tracts['publicTrans_class'] = merged_tracts[['Pct_PublicTrans']].apply(classifier_public_trans)
merged_tracts['rentBurden_class'] = merged_tracts[['Pct_RentBurden']].apply(classifier_rent_burden)
merged_tracts['belowPov_class'] = merged_tracts[['Pct_BelowPov']].apply(classifier_below_pov)

# Check the head of the dataframe to see the new columns
print(merged_tracts[['publicTrans_class', 'rentBurden_class', 'belowPov_class']].head())

In [ ]:
# Create the 'housing_transit_index' column
merged_tracts['housing_transit_index'] = (
    merged_tracts['publicTrans_class'] + 
    merged_tracts['rentBurden_class'] + 
    merged_tracts['belowPov_class'] + 
    merged_tracts['nonWhite_class']
)

# Plot the 'housing_transit_index' using equal intervals
fig, ax = plt.subplots(figsize=(10, 10))
merged_tracts.plot(column="housing_transit_index", scheme="Equal_Interval", k=5, cmap="YlGn", linewidth=0, legend=True, ax=ax)

# Use tight layout
plt.tight_layout()
plt.title('Housing and Transit Vulnerability Index in LA County')
ax.axis('off')

# Place legend outside of plot
ax.get_legend().set_bbox_to_anchor((1.5, 1))
plt.show()
# Save the plot to a PNG file
plt.savefig('housing_transit_vulnerability_index.png')

In [ ]:
import folium
import contextily as ctx
from folium.plugins import HeatMap

# Create a folium map centered around LA County
m = folium.Map(location=[34.0522, -118.2437], zoom_start=8, tiles='cartodbpositron')

# Display the map
folium.Choropleth(
    geo_data=merged_tracts,
    name='choropleth',
    data=merged_tracts,
    columns=['GEOID', 'housing_transit_index'],
    key_on='feature.properties.GEOID',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Housing and Transit Vulnerability Index',
    bins=5,  # Set the number of bins for equal interval classification
    reset=True
).add_to(m)

# Save the map as an HTML file
m.save('housing_transit_vulnerability_index.html')

m 

A-haa, okay so we can see where suitable places for us with our criteria seem to be located in the county. What policy recommendations could we make with this index?

**Other examples and resources**

- Notebook source from [AutoGIS](https://autogis-site.readthedocs.io/en/2019/notebooks/L4/reclassify.html)
- CDC [Social Vulnerability Index](https://www.atsdr.cdc.gov/place-health/php/svi/svi-interactive-map.html)